# Environment Preparation

In [1]:
import os
import sys
# set environment variable
os.environ['SPARK_HOME'] = "/home/bin_yin/tmp/spark-2.0.2-bin-hadoop2.7/"
os.environ['PYSPARK_SUBMIT_ARGS'] = "--master local[2] pyspark-shell" 
# we can check by os.environ.get("SPARK_HOME")

# Init sc
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="myAppName")

# 1. Data Description

**u.user**

u'1|24|M|technician|85711'
* [0]: users id
* [1]: age
* [2]: gender
* [3]: occupation
* [4]: zipcode


**u.item**

u'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0'

* [0]: movie id
* [1]: movie title
* [2]: release date
* [3]: video release date
* [4]: IMDb URL
* unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western


**u.data**

u'196\t242\t3\t881250949'
* [0]: user id
* [1]: item id
* [2]: rating
* [3]: timestamp

# 2. Train & Predict

In [2]:
# get data
raw_data = sc.textFile('ml-100k/u.data')
raw_ratings = raw_data.map(lambda line:line.split('\t'))

ratings = raw_ratings.map(  lambda f:( int(f[0]), int(f[1]), float(f[2]))  )
ratings.first() # 用户ID，电影ID，得分

(196, 242, 3.0)

In [3]:
# check ratings data
import numpy as np
np.array(ratings.collect()).shape# 矩阵维度：(100000, 3)

(100000, 3)

In [8]:
%%time
# train model
from  pyspark.mllib.recommendation import ALS
model = ALS.train(ratings, rank=50, iterations=10, lambda_=0.01)

# 核心原理就是把ratings矩阵分解（协同过滤），类似为每个用户训练一个线性回归模型
# rank：线性模型维度

CPU times: user 20 ms, sys: 2 ms, total: 22 ms
Wall time: 5.58 s


In [7]:
# predict

model.predict(789, 123) # 用户789对电影123的打分是3.107

3.107056655027367

In [9]:
sc.stop()# Stop Spark